In [1]:
import pickle
import re
from nltk.tokenize import TweetTokenizer
import numpy as np
from sklearn import svm
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV 
from preprocessing.tweet import Tweet
from nltk.data import load


In [2]:
train = np.asarray(pickle.load(open("preprocessing/train_set", "rb")))
test = np.asarray(pickle.load(open("preprocessing/test_set", "rb")))


In [3]:
y_train = [int(tweet.valence) for tweet in train]
y_test = [int(tweet.valence) for tweet in test]


In [4]:
def get_num_of_hashtags(data_set):
    X = np.zeros((len(data_set), 1))
    
    for i, tweet in enumerate(data_set):
        X[i, 0] = np.asarray(len(tweet.hashtags))
        
    return X


In [5]:
def get_num_of_mult_punctuation(data_set):
    X = np.zeros((len(data_set), 1))

    for i, tweet in enumerate(data_set):
        X[i, 0] = len(re.findall(r"[?!]{2,}", tweet.raw))
        
    return X


In [6]:
def get_num_pos_tags(train, test):
    vocabulary = list(load('help/tagsets/upenn_tagset.pickle'))
    pos_tags_train = [tweet.pos_tags for tweet in train]
    pos_tags_test = [tweet.pos_tags for tweet in test]

    vectorizer = CountVectorizer(vocabulary=vocabulary, tokenizer=lambda doc: doc, lowercase=False)

    train_vector = vectorizer.transform(pos_tags_train)
    test_vector = vectorizer.transform(pos_tags_test)
   
    return np.asarray(train_vector.toarray()), np.asarray(test_vector.toarray())



In [7]:
# TODO: split for train and test and return vectorizer


def get_ngram_vectors(train, test, method="word", upper=3, lower=1):
    vectorizer = CountVectorizer(ngram_range=(lower, upper), tokenizer=TweetTokenizer().tokenize, 
                                 analyzer=method)
    
    raw_train = [tweet.raw for tweet in train]
    raw_test = [tweet.raw for tweet in test]
    
    train_vector = vectorizer.fit_transform(raw_train)
    test_vector = vectorizer.transform(raw_test)

    return np.asarray(train_vector.toarray()), np.asarray(test_vector.toarray())


In [8]:
def extract_features(train, test):
    word_ngram_upper_bound = 4
    char_ngram_upper_bound = 5
    char_ngram_lower_bound = 3

    pos_vec_train, pos_vec_test = get_num_pos_tags(train, test)
    train_word_vectors, test_word_vectors = get_ngram_vectors(
        train, test, "word", word_ngram_upper_bound)
    train_char_vectors, test_char_vectors = get_ngram_vectors(
        train, test, "char", char_ngram_upper_bound, char_ngram_lower_bound)
    
    train_features = np.concatenate([get_num_of_hashtags(train),
                                     get_num_of_mult_punctuation(train),
                                     pos_vec_train,
                                     train_word_vectors,
                                     train_char_vectors], axis=1)

    test_features = np.concatenate([get_num_of_hashtags(test),
                                    get_num_of_mult_punctuation(test),
                                    test_word_vectors,
                                    test_char_vectors
                                    ], axis=1)
    
    return train_features, test_features


In [9]:
def transform_to_pos_neg(np_array):
    np_array[np_array < 0] = -1
    np_array[np_array > 0] = 1
    
    return np_array


In [10]:
def evaluate_model(model, X, y):
    prediction = model.predict(X)

    acc = metrics.accuracy_score(y, prediction)
    print("Accuracy:", acc)

    acc_interval = acc + metrics.accuracy_score(y + 1, prediction) \
                   + metrics.accuracy_score(y - 1, prediction)
    print("Accuracy for small interval:", acc_interval)  # Accuracy if predicted value is +-1

    acc_dir = metrics.accuracy_score(transform_to_pos_neg(y), transform_to_pos_neg(prediction))
    print("Accuracy for right direction:", acc_dir)


In [11]:
X_train, X_test = extract_features(train, test)


## Training without Grid Search

In [12]:
# TODO: normalization? 

lin_svm = svm.LinearSVC()
lin_svm.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [13]:
evaluate_model(lin_svm, X_test, y_test)

## Training with Grid Search

In [15]:
cs = [0.001, 0.01, 0.1, 1, 10]
grid = {'C': cs}

grid_search = GridSearchCV(svm.LinearSVC(), grid, cv=5, verbose=10)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] C=0.001 .........................................................


[CV] .......................... C=0.001, score=0.309322, total=   1.0s
[CV] C=0.001 .........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV] .......................... C=0.001, score=0.360169, total=   0.9s
[CV] C=0.001 .........................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.3s remaining:    0.0s


...

[CV] ............................. C=10, score=0.331915, total=   5.0s
[CV] C=10 ............................................................


[CV] ............................. C=10, score=0.337607, total=   3.3s
[CV] C=10 ............................................................


[CV] ............................. C=10, score=0.296137, total=   3.9s


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  1.1min finished


{'C': 0.01}


In [15]:
evaluate_model(grid_search, X_test, y_test)


Accuracy: 0.145
Accuracy for small interval: 0.635
Accuracy for right direction: 0.505
